# LinkedIn Data Analyst Job Search

This is a personal project made to improve my web scraping abilities using the an automated bot through the selenium python libaray. 

In addition, it was made to survey the data science/analyst landscape as required by employers as advertised on LinkedIn. I hoped to guaged the potential programming languages and tools which dominate the space. Because this is based on Australian listings, I hope to also visualise geographical results. What do employers expect? Is Python or R most required? These are the questions we as data science students tend to ask ourselves. This tool perhaps provides some light into the pool of uncertainty.

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time 
from selenium.webdriver.common.by import By
from statistics import mode
from collections import Counter
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import requests
import en_core_web_sm
nlp = en_core_web_sm.load()
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Specify path file location and service

PATH = "C:\\Users\\Ivan Shamoon\\Desktop\\chromedriver.exe"
url = 'https://au.linkedin.com/jobs/data-analyst-jobs?position=1&pageNum=0'

 # Set up driver through the driver located in the path. 
 # The driver is set up to lauch a proxy web browser when ran.
 # The driver then must be given website input to lauch towards a desired location.
 # Expand window size for full website capabilities

driver = webdriver.Chrome(executable_path = PATH)
driver.get(url)
driver.maximize_window()

C:\Users\Ivan Shamoon\AppData\Local\Temp\ipykernel_10756\2930643640.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = PATH)


## Define Function

stop_words_clean: remove stop words and punctuation from the job descriptions

augment_tool: encoder for inputed list or string. ie input 'sql' inspect whether current description contains the word 'sql': 1 if yes 0 if no

In [ ]:
# Tool that records the occurance of inputed lists and determines for each row if each element in the
# Inputed list occurs in the dataframe's row.

def augment_tool(df, tools):
    df[tools] = 'NA'
    for item in tools:
        index = 0
        for desc in df['job_desc']:
            if item in desc.lower().split(' '):
                df[item][index] = 1
            else:
                df[item][index] = 0
            index = index + 1

In [ ]:
# Outputs a dictionary for each element in inputed data where stopwords are removed.
# The returned values are split

def stop_words_clean(job_desc):

    words = job_desc
    filtered_sentences = {}
    index = 0

    for synopsis in job_desc:

        # Initialise with nlp    
        my_doc = nlp(synopsis.lower())
        token_list = []
        
        # Convert all words in each synopsis to text
        for word in my_doc:
            token_list.append(word.text)

        # For each word in a synopsis, determine whether current element is a stopword
        # If not, append word into dictionary keyed by index
        for word in token_list:
            lexeme = nlp.vocab[word]

            if index not in filtered_sentences:
                if lexeme.is_stop == False:
                    filtered_sentences[index] = [word]

            else:
                if lexeme.is_stop == False:
                    filtered_sentences[index].append(word)

        # Move to next synopsis
        index = index + 1

    # Join each element in the dictionary whose entries are separated
    for key in filtered_sentences:
        filtered_sentences[key] = ' '.join(filtered_sentences[key]) 
    
    # Expand dictionary entries into a list and remove punctuation
    filtered_sentences = pd.Series(filtered_sentences)
    filtered_sentences = filtered_sentences.str.replace(r'[^\w\s]+', '')

    # Remove last paragraph (it is irrelavent)
    loop = []
    index = 0
    for item in filtered_sentences:
        element = item.split('\n\n')
        length = len(element)
        filtered_sentences[index] = element[0:length-1]
        index = index + 1

    index = 0
    for item in filtered_sentences:
        filtered_sentences[index] = ' '.join(filtered_sentences[index])
        index = index + 1
 
    return filtered_sentences

## Issues that needed to be addressed

Unlike static pages where the web sacrping tool BeautifulSoup excels in, to gather the most information and to utilize the clicking options offered by LinkedIn, a most robust tool in the form of Selenium must be used. 

The page layout of LinkedIn job listings is not static. It requires the user to individually click on listings on the side-board for a pop up with specific information about the job to show up. Three things to note about this:

* The layout can only be accessed when the window size is large enough to offer the information. This is why the driver's window is set to the maximum size
* There needs to be a wait period when clicking due to internet buffering or slowness
* What happens when gitches in the website occurs and no pop up shows despite clicking on it. The try/except functions helps to pass through the loop when this occurs. Therefore the possibility of errors cannot hault the code from running.

In [ ]:
main_block = driver.find_element(By.CLASS_NAME, 'two-pane-serp-page__results-list')

for i in range(0,7):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)

for i in range(0,10):
    more_job_button = main_block.find_element(By.TAG_NAME, 'button')
    more_job_button.click()
    time.sleep(3)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [ ]:
first_two = 0
job_desc = []
job_title = []
job_indus = []
job_location = []
job_employ = []

# Identifies main block for job listings
main_block = driver.find_element(By.CLASS_NAME, 'two-pane-serp-page__results-list')

# Select all individual blocks from job listing
job_block = main_block.find_elements(By.TAG_NAME, 'li')

# Go through each item in the element
for index in range(0,len(job_block)):

    # Identify block
    block = job_block[index]

    # Clicks the job and opens the side bar avaliability
    block.click()

    # Wait for contents to completely load
    time.sleep(2)

    try:
        # Top portion of select job option
        top_card = driver.find_element(By.CLASS_NAME, 'two-pane-serp-page__detail-view')

        # With loaded contents select the side bar element
        show_more = driver.find_element(By.CLASS_NAME,'decorated-job-posting__details')

        # Get Title    
        title = top_card.find_element(By.CLASS_NAME, 'topcard__link')

        # Get location
        location = top_card.find_element(By.XPATH, "//span[@class = 'topcard__flavor topcard__flavor--bullet']")

        # Get bottom card 
        bottom_card = top_card.find_elements(By.XPATH, "//span[@class = 'description__job-criteria-text description__job-criteria-text--criteria']")
            
        # Get industry 
        industry = bottom_card[3]

        # Get employment type
        employment = bottom_card[1]
            
        # From side bar element click the show more button for full description
        show_more_button = show_more.find_element(By.TAG_NAME, 'button')
        show_more_button.click()

        # Wait for loaded contents
        time.sleep(2)

        # Append items
        job_desc.append(show_more.text)
        job_title.append(title.text)
        job_indus.append(industry.text)
        job_location.append(location.text)
        job_employ.append(employment.text)
    except:
        continue




In [ ]:
data = pd.DataFrame({'job_title': job_title,
                    'job_desc': job_desc,
                    'job_location': job_location,
                    'job_indus': job_indus,
                    'employ_type': job_employ})


## Cleaning the Data and Transformation

In [6]:
# Strips \n element from each job description.
# Removes punctuation from each element for future word identification easibility
index = 0
for item in data['job_desc']:
    element = item.split('\n')
    element = list(map(str.strip, element))
    element = ' '.join(element)
    data['job_desc'][index] = element
    index = index + 1

data['job_desc'] = data['job_desc'].str.replace(r'[^\w\s]+', '')

C:\Users\Ivan Shamoon\AppData\Local\Temp\ipykernel_10756\264959438.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['job_desc'][index] = element
C:\Users\Ivan Shamoon\AppData\Local\Temp\ipykernel_10756\264959438.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data['job_desc'] = data['job_desc'].str.replace(r'[^\w\s]+', '')


This is not an exhaustive list of tools, but is information gathered from the internet and surveying a lot of job listings and determining common words, languages, and BI tools in most listings within Australia.

In [ ]:
prog_lan = ['sql','r','python','java','matlab','javascript','mongodb','nosql','mysql','c#','c++']
data_tools = ['powerbi', 'power', 'microstrategy','looking', 'sap', 'sisense','tibco', 'qlik', 'qlikwise', 'sas','jupyter', 'agile','bigquery' , 'tableau', 'google','IBM','chartio','excel','alteryx','word','powerpoint','shiny','lambda','azure','teradata']

augment_tool(data,prog_lan)
augment_tool(data,data_tools)

In [ ]:
# Get first element from job industry
index = 0
for item in data['job_indus']:
    element = item.split(',')[0]
    data['job_indus'][index] = element
    index = index + 1

# Set column stype to string
data['job_indus'] = data['job_indus'].astype(str)

Because there are a lot of industries with many of them overlapping but not sematically, it is a good idea to cohort them into their general field. Otherwise many information in the visualisation will be lost in the technicality.

In [ ]:
# Streamline the job industries into its category

for index, item in enumerate(data['job_indus']):
    if 'retail' in item.lower():
        data['job_indus'][index] = 'Retail'
    elif any(word in item.lower() for word in ['pharam' ,'medic', 'biot','health']):
        data['job_indus'][index] = 'Health'
    elif any(word in item.lower() for word in ['insur', 'financ','inves', 'estate']):
        data['job_indus'][index] = 'Financial Services'
    elif any(word in item.lower() for word in ['truck', 'transp','oil', 'travel','automo','airlin']):
        data['job_indus'][index] = 'Transportation'
    elif any(word in item.lower() for word in ['comput','software', 'techno','applianc']):
        data['job_indus'][index] = 'Software Development'
    elif any(word in item.lower() for word in ['engineering','manufa','construc','mining','building','build','environ']):
        data['job_indus'][index] = 'Engineering'
    elif any(word in item.lower() for word in ['legal','govern']):
        data['job_indus'][index] = 'Government'
    elif any(word in item.lower() for word in ['telecom','adver','entertain', 'media','communi']):
        data['job_indus'][index] = 'Advertisement'
    elif any(word in item.lower() for word in ['human','education','e-learn','public','social']):
        data['job_indus'][index] = 'Human Services'
    elif any(word in item.lower() for word in ['it', 'consulting','resear']):
        data['job_indus'][index] = 'IT Consulting'


Clean the location and designate them into the dataframe

In [ ]:
# Attempt to retreive second element from job location column as the majority of the states are found in there
# Else retreive the whole element
capital =  ['sydney', 'melbourne', 'hobart','brisbane','adelaide','perth']
state = ['New South Wales',' Victoria','Tasmania','Queesland','South Australia','Western Australia']

for index, item in enumerate(data['job_location']):
    try:
        data['job_location'][index] = item.split(',')[1].strip()
    except:
        if capital[0] in item.lower():
            data['job_location'][index] = state[0]
        elif capital[1] in item.lower():
            data['job_location'][index] = state[1]
        elif capital[2] in item.lower():
            data['job_location'][index] = state[2]
        elif capital[3] in item.lower():
            data['job_location'][index] = state[3]
        elif capital[4] in item.lower():
            data['job_location'][index] = state[4]
        elif capital[5] in item.lower():
            data['job_location'][index] = state[5]

In [ ]:
data['job_location'].value_counts()

In [ ]:
data.to_csv('linked_scrap.csv')
data = pd.read_csv('linkedin_scrap.csv')

In [18]:
prog_lan = ['sql','r','python','java','matlab','javascript','mongodb','nosql','mysql','c#','c++']
data_tools = ['powerbi', 'power', 'microstrategy', 'sap', 'sisense','tibco', 'qlik', 'qlikwise', 'sas','jupyter', 'agile','bigquery' , 'tableau', 'google','IBM','chartio','excel','alteryx','word','powerpoint','shiny','lambda','azure','teradata']


In [16]:
data[prog_lan].sum()

sql           256
r              77
python        118
java           11
matlab          1
javascript      6
mongodb         2
nosql           6
mysql           1
c#              0
c++             0
dtype: int64

In [19]:
data[data_tools].sum()

powerbi           36
power            148
microstrategy      4
sap               19
sisense            1
tibco              0
qlik              20
qlikwise           0
sas               24
jupyter            4
agile             76
bigquery          14
tableau           97
google            44
IBM                0
chartio            0
excel            115
alteryx           27
word              15
powerpoint        11
shiny              2
lambda             3
azure             58
teradata          18
dtype: int64